<a href="https://colab.research.google.com/github/LCaravaggio/Cosas_Sueltas/blob/main/Merge_de_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c examen-2-ap-itba


import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

Mounted at /content/drive
 94% 49.0M/52.0M [00:02<00:00, 25.5MB/s]
100% 52.0M/52.0M [00:02<00:00, 21.6MB/s]


In [5]:
df_train=pd.read_csv("datos.csv", sep=",")
df_test=pd.read_csv("resultados.csv", sep=",")
sol=pd.read_csv("solucion.csv", sep=",")

<ipython-input-5-f741a21bedf9>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train=pd.read_csv("datos.csv", sep=",")


In [6]:
dos=pd.merge(df_test, sol, left_index=True, right_index=True)

In [7]:
todo=pd.concat([df_train, dos])

In [8]:
df_train.columns

Index(['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_x', 'directors', 'writers',
       'seasonNumber', 'episodeNumber', 'ordering', 'language', 'types',
       'attributes', 'isOriginalTitle', 'adult', 'budget', 'genres_y',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'video', 'id'],
      dtype='object')

In [9]:
dos.columns

Index(['numVotes', 'titleType', 'isAdult', 'startYear', 'endYear',
       'runtimeMinutes', 'genres_x', 'directors', 'writers', 'seasonNumber',
       'episodeNumber', 'ordering', 'language', 'types', 'attributes',
       'isOriginalTitle', 'adult', 'budget', 'genres_y', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'video', 'id_x', 'id_y', 'averageRating'],
      dtype='object')

In [10]:
todo.columns

Index(['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_x', 'directors', 'writers',
       'seasonNumber', 'episodeNumber', 'ordering', 'language', 'types',
       'attributes', 'isOriginalTitle', 'adult', 'budget', 'genres_y',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'video', 'id', 'id_x', 'id_y'],
      dtype='object')

In [11]:
todo=todo.replace('Game-Show',0)
todo=todo.replace('\\n',0)
todo=todo.replace('Reality-TV',0)
todo=todo.replace('\\N',0)

In [12]:
def replaceN (df,columna):
  df[columna] = df[columna].replace('\\\\N','hola').replace('hola',np.nan)

replaceN(todo,'endYear')
replaceN(todo,'runtimeMinutes')
replaceN(todo,'genres_x')
replaceN(todo,'startYear')
replaceN(todo,'directors')
replaceN(todo,'writers')
replaceN(todo,'seasonNumber')
replaceN(todo,'episodeNumber')
replaceN(todo,'language')
replaceN(todo,'types')
replaceN(todo,'attributes')

In [13]:
todo=todo.drop(columns=['id_x', 'id_y'])

In [14]:
todo=todo.drop(columns=['id'])

In [15]:
todo = todo[~todo['runtimeMinutes'].isnull()]
todo['runtimeMinutes']=todo['runtimeMinutes'].replace('Game-Show',0)
todo['runtimeMinutes']=todo['runtimeMinutes'].astype('int16')

In [16]:
todo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1221927 entries, 0 to 321931
Data columns (total 31 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   averageRating         1221927 non-null  float64
 1   numVotes              1221927 non-null  int64  
 2   titleType             1221927 non-null  object 
 3   isAdult               1221927 non-null  float64
 4   startYear             1221927 non-null  object 
 5   endYear               1221927 non-null  object 
 6   runtimeMinutes        1221927 non-null  int16  
 7   genres_x              1221925 non-null  object 
 8   directors             1221927 non-null  object 
 9   writers               1221927 non-null  object 
 10  seasonNumber          547631 non-null   object 
 11  episodeNumber         547631 non-null   object 
 12  ordering              463440 non-null   float64
 13  language              463440 non-null   object 
 14  types                 463440 non-nu

In [17]:
todo=todo.drop(columns=['types'])

In [18]:
todo=todo.drop(columns=['overview'])

In [24]:
todo=todo.drop(columns=['release_date'])

In [25]:
todo.columns

Index(['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_x', 'directors', 'writers',
       'seasonNumber', 'episodeNumber', 'ordering', 'language', 'attributes',
       'isOriginalTitle', 'adult', 'budget', 'genres_y', 'original_language',
       'popularity', 'production_companies', 'production_countries', 'revenue',
       'runtime', 'status', 'tagline', 'video'],
      dtype='object')

In [26]:
from sklearn.model_selection import train_test_split

In [28]:
datos, sol, AR_datos, AR_sol = train_test_split(todo, todo.averageRating, test_size=0.20, random_state=100)

In [29]:
datos=datos.reset_index()

In [30]:
sol=sol.reset_index()

In [31]:
AR_datos=AR_datos.reset_index()

In [32]:
AR_sol=AR_sol.reset_index()

In [33]:
datos=datos.drop(columns=['index'])

In [34]:
sol=sol.drop(columns=['index', 'averageRating'])

In [35]:
AR_sol=AR_sol.drop(columns=['index'])

In [36]:
AR_datos=AR_datos.drop(columns=['index'])

In [37]:
datos.to_csv('origen.csv')

In [38]:
sol.to_csv('testear.csv')

In [39]:
AR_sol['Usage']=""

In [40]:
len(AR_sol['Usage'])

244386

In [47]:
AR_sol['Usage'][:75000]='Public'

<ipython-input-47-8fd849cc6c49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AR_sol['Usage'][:75000]='Public'


In [48]:
AR_sol['Usage'][75000:]='Private'

<ipython-input-48-ae1d1aebac8e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AR_sol['Usage'][75000:]='Private'


In [49]:
AR_sol.to_csv('solucion_2023.csv')

In [50]:
from google.colab import files
files.download('solucion_2023.csv')
files.download('testear.csv')
files.download('origen.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>